In [ ]:
#셀레늄 설치
!pip install selenium
!pip install beautifulsoup4
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and plat

In [ ]:
# 모듈 임포트
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
from newspaper import Article
from datetime import datetime
import pandas as pd

In [ ]:
# 옵션 설정
options = Options()
options.add_argument('--headless')  # 헤드리스 모드
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

1.WORLD MARKET

In [ ]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.get('https://www.cnbc.com/world-markets/') # 브라우저 불러오기

In [ ]:
# Load More 버튼을 반복적으로 클릭하여 더 많은 기사 로드
while True:
    try:
        # iframe 숨기기
        iframes = browser.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            browser.execute_script("arguments[0].style.visibility='hidden';", iframe)

        # 페이지 스크롤
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Load More 버튼이 나타날 때까지 대기
        load_more_button = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'icon-arrow-down-readmore'))
        )

        # ActionChains를 사용하여 버튼이 가려지지 않도록 조정
        actions = ActionChains(browser)
        actions.move_to_element(load_more_button).perform()
        time.sleep(2)  # 페이지 로드 대기
        load_more_button.click()
        time.sleep(5)  # 페이지 로드 대기
    except Exception as e:
        print(f"No more articles to load or an error occurred: {e}")
        break
print('All articles loaded.')

No more articles to load or an error occurred: Message: 
Stacktrace:
#0 0x5b5b8ff1bc7a <unknown>
#1 0x5b5b8fbfee2c <unknown>
#2 0x5b5b8fc4b661 <unknown>
#3 0x5b5b8fc4b751 <unknown>
#4 0x5b5b8fc8ff64 <unknown>
#5 0x5b5b8fc6e5ed <unknown>
#6 0x5b5b8fc8d303 <unknown>
#7 0x5b5b8fc6e363 <unknown>
#8 0x5b5b8fc3e247 <unknown>
#9 0x5b5b8fc3eb9e <unknown>
#10 0x5b5b8fee224b <unknown>
#11 0x5b5b8fee62f1 <unknown>
#12 0x5b5b8fecdafe <unknown>
#13 0x5b5b8fee6e52 <unknown>
#14 0x5b5b8feb279f <unknown>
#15 0x5b5b8ff0b638 <unknown>
#16 0x5b5b8ff0b810 <unknown>
#17 0x5b5b8ff1adac <unknown>
#18 0x7c12fa4c9ac3 <unknown>

All articles loaded.


In [ ]:
# URL 수집을 위한 BeautifulSoup 사용
soup = BeautifulSoup(browser.page_source, 'html.parser')

# URL 리스트 불러오기
url_lst = []
contents = soup.find_all('a', class_='Card-title')
for content in contents:
    url_lst.append(content['href'])
url_lst = list(set(url_lst))  # 중복 URL 제거
print(f"Total URLs collected: {len(url_lst)}")

Total URLs collected: 200


In [ ]:
url_lst[:]

['https://www.cnbc.com/2024/07/08/oil-little-changed-as-investors-eye-impact-of-storm-beryl-on-texas.html',
 'https://www.cnbc.com/2024/06/17/europe-stocks-markets-earnings-data.html',
 'https://www.cnbc.com/2024/06/06/gold-gains-as-us-dollar-yields-slip-focus-on-jobs-data.html',
 'https://www.cnbc.com/2024/05/31/gold-flat-ahead-of-us-data-but-set-for-4th-straight-monthly-gain.html',
 'https://www.cnbc.com/2024/06/27/gold-prices-lingers-near-two-week-low-as-traders-eye-us-inflation-data.html',
 'https://www.cnbc.com/2024/06/14/asia-markets.html',
 'https://www.cnbc.com/2024/06/20/european-markets-stocks-news-bank-of-england-rate-decision.html',
 'https://www.cnbc.com/2024/06/04/european-financial-markets-stocks-news-data-and-earnings.html',
 'https://www.cnbc.com/2024/07/01/european-markets-react-to-france-election-german-inflation-data.html',
 'https://www.cnbc.com/2024/06/19/asia-pacific-markets.html',
 'https://www.cnbc.com/2024/05/28/oil-prices-steady-as-investors-await-inflation-d

In [ ]:
# 브라우저 종료
browser.quit()

In [ ]:
# 기사 수집 함수
def get_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "text": article.text,
        "publish_date": article.publish_date
    }

In [ ]:
# 기사 수집
collected_articles = []

In [ ]:
for url in url_lst:
    try:
        content = get_article_content(url)
        publish_date = content["publish_date"]

        # 기사 데이터 추가
        collected_articles.append({
            "date": publish_date,
            "title": content["title"],
            "text": content["text"]
        })
    except Exception as e:
        print(f"Failed to process article at {url}: {e}")

In [ ]:
# 수집된 기사 출력
for article in collected_articles:
    print(f"Date: {article['date']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['text'][:200]}...")  # 본문은 길기 때문에 앞부분만 출력
    print()

Date: 2024-07-08 00:00:00
Title: Oil slips as Gaza talks ease supply worries; Hurricane Beryl in focus
Content: Oil prices slid on Monday after rising for four weeks, as the prospect of a ceasefire deal in Gaza eased tensions in the Middle East, while investors assessed potential disruption to U.S. energy suppl...

Date: 2024-06-17 00:00:00
Title: Europe stocks close slightly higher; France's CAC 40 makes cautious rebound
Content: Commuters crossing a junction near the Bank of England (BOE), left, in the City of London, UK, on Wednesday, May 8, 2024. Bank of England policymakers appear the most divided since they brought their ...

Date: 2024-06-06 00:00:00
Title: Gold hits two-week high, payrolls data in focus
Content: Gold firmed on Thursday as the dollar and Treasury yields retreated on rising bets that U.S interest rate cuts may start as early as September, while investors awaited the U.S. non-farm payrolls data....

Date: 2024-05-31 00:00:00
Title: Gold posts fourth monthly rise o

In [ ]:
df = pd.DataFrame(collected_articles)

In [ ]:
df.head()

,date,title,text
0,2024-07-08,Oil slips as Gaza talks ease supply worries; H...,Oil prices slid on Monday after rising for fou...
1,2024-06-17,Europe stocks close slightly higher; France's ...,Commuters crossing a junction near the Bank of...
2,2024-06-06,"Gold hits two-week high, payrolls data in focus",Gold firmed on Thursday as the dollar and Trea...
3,2024-05-31,Gold posts fourth monthly rise on Fed rate cut...,One kilogram gold bullion at the YLG Bullion I...
4,2024-06-27,Gold gains 1% on dollar weakness as focus turn...,Gold prices steadied on Thursday after falling...


In [ ]:
df.tail()

,date,title,text
195,2024-06-20,Asia-Pacific markets are mostly lower as China...,China kept its one- and five-year loan prime r...
196,2024-05-16,Oil prices rise on moderate U.S. inflation dat...,An oil pumpjack is seen near a field of wind t...
197,2024-06-19,European markets close slightly lower; FTSE ti...,The Stoxx 600 index ended the session down 0.1...
198,2024-06-18,Europe stocks close higher; Hargreaves Lansdow...,A stock trader looks at his monitors in the tr...
199,2024-05-28,European markets close lower as travel and lei...,European Central Bank Governing Council member...


In [ ]:
# 날짜 순으로 정렬
df2 = df.sort_values(by='date')

In [ ]:
df2.head()

,date,title,text
113,2024-05-16,Dollar firm as investors await Fed guidance,The dollar was on the front foot on Wednesday ...
121,2024-05-16,Asia markets rise after Wall Street soars on s...,Workers install solar panels during a SunPower...
28,2024-05-16,Europe stocks close lower as earnings dampen r...,European stocks closed lower Thursday as earni...
102,2024-05-16,"Gold eases as dollar gains, Fed rate cut bets ...",old prices edged higher on Wednesday after sof...
196,2024-05-16,Oil prices rise on moderate U.S. inflation dat...,An oil pumpjack is seen near a field of wind t...


In [ ]:
df2.tail()

,date,title,text
162,2024-07-16,Asia-Pacific markets mixed after Fed Chair Pow...,China's biggest policy meeting in six years ki...
132,2024-07-16,Gold drifts higher as U.S. rate cut optimism b...,"A five hundred gram gold bar, left, and a a on..."
142,2024-07-16,Europe heads for mixed open as markets digest ...,U.S. Federal Reserve Chair Jerome Powell speak...
181,2024-07-16,Oil prices tick down on worries about Chinese ...,Oil prices edged lower on Tuesday on worries a...
90,2024-07-16,Dollar drifts as Powell comments bring Septemb...,The dollar fell slightly on Monday after comme...


In [ ]:
# 구글 드라이브 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 엑셀 파일을 Google Drive의 특정 경로에 저장
excel_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_Worldmarket.xlsx'
df2.to_excel(excel_path, index=False)

2. ECONOMY

In [ ]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.get('https://www.cnbc.com/economy/') # 브라우저 불러오기

In [ ]:
# Load More 버튼을 반복적으로 클릭하여 더 많은 기사 로드
while True:
    try:
        # iframe 숨기기
        iframes = browser.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            browser.execute_script("arguments[0].style.visibility='hidden';", iframe)

        # 페이지 스크롤
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Load More 버튼이 나타날 때까지 대기
        load_more_button = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'icon-arrow-down-readmore'))
        )

        # ActionChains를 사용하여 버튼이 가려지지 않도록 조정
        actions = ActionChains(browser)
        actions.move_to_element(load_more_button).perform()
        time.sleep(2)  # 페이지 로드 대기
        load_more_button.click()
        time.sleep(5)  # 페이지 로드 대기
    except Exception as e:
        print(f"No more articles to load or an error occurred: {e}")
        break
print('All articles loaded.')

No more articles to load or an error occurred: Message: 
Stacktrace:
#0 0x5b7fc724ec5a <unknown>
#1 0x5b7fc6f31e2c <unknown>
#2 0x5b7fc6f7e661 <unknown>
#3 0x5b7fc6f7e751 <unknown>
#4 0x5b7fc6fc2f64 <unknown>
#5 0x5b7fc6fa15ed <unknown>
#6 0x5b7fc6fc0303 <unknown>
#7 0x5b7fc6fa1363 <unknown>
#8 0x5b7fc6f71247 <unknown>
#9 0x5b7fc6f71b9e <unknown>
#10 0x5b7fc721522b <unknown>
#11 0x5b7fc72192d1 <unknown>
#12 0x5b7fc7200ade <unknown>
#13 0x5b7fc7219e32 <unknown>
#14 0x5b7fc71e577f <unknown>
#15 0x5b7fc723e618 <unknown>
#16 0x5b7fc723e7f0 <unknown>
#17 0x5b7fc724dd8c <unknown>
#18 0x79191af7dac3 <unknown>

All articles loaded.


In [ ]:
# URL 수집을 위한 BeautifulSoup 사용
soup = BeautifulSoup(browser.page_source, 'html.parser')

# URL 리스트 불러오기
url_lst = []
contents = soup.find_all('a', class_='Card-title')
for content in contents:
    url_lst.append(content['href'])
url_lst = list(set(url_lst))  # 중복 URL 제거
print(f"Total URLs collected: {len(url_lst)}")

Total URLs collected: 201


In [ ]:
url_lst[:]

['https://www.cnbc.com/2024/05/25/the-fed-probably-wont-deliver-any-interest-rate-cuts-this-summer-.html',
 'https://www.cnbc.com/2024/07/11/uk-economic-growth-may-2024.html',
 'https://www.cnbc.com/2024/06/07/spief-2024-russia-showcases-economy-as-ukraine-war-takes-a-back-seat.html',
 'https://www.cnbc.com/2024/03/12/cpi-inflation-report-february-2024-.html',
 'https://www.cnbc.com/2024/04/26/pce-inflation-march-2024-key-fed-inflation-measure-rose-2point8percent.html',
 'https://www.cnbc.com/2024/07/13/the-housing-market-explained-in-6-charts.html',
 'https://www.cnbc.com/2024/04/25/slowing-growth-plus-stubborn-inflation-add-up-to-a-headache-for-fed-officials.html',
 'https://www.cnbc.com/2024/05/31/euro-zone-inflation-may-2024.html',
 'https://www.cnbc.com/2024/05/03/jobless-rates-rise-in-april-for-all-race-groups-except-black-americans.html',
 'https://www.cnbc.com/2024/05/02/wall-street-is-confused-and-divided-over-how-many-times-the-fed-will-cut-rates-this-year.html',
 'https://ww

In [ ]:
# 브라우저 종료
browser.quit()

In [ ]:
# 기사 수집 함수
def get_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "text": article.text,
        "publish_date": article.publish_date
    }

In [ ]:
# 기사 수집
collected_articles = []

In [ ]:
for url in url_lst:
    try:
        content = get_article_content(url)
        publish_date = content["publish_date"]

        # 기사 데이터 추가
        collected_articles.append({
            "date": publish_date,
            "title": content["title"],
            "text": content["text"]
        })
    except Exception as e:
        print(f"Failed to process article at {url}: {e}")

In [ ]:
# 수집된 기사 출력
for article in collected_articles:
    print(f"Date: {article['date']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['text'][:200]}...")  # 본문은 길기 때문에 앞부분만 출력
    print()

Date: 2024-05-25 00:00:00
Title: The Fed probably won't be delivering any interest rate cuts this summer
Content: Traders work on the floor of the New York Stock Exchange during morning trading on May 24, 2024 in New York City. Michael M. Santiago | Getty Images

Investors likely will have to sweat out a summer d...

Date: 2024-07-11 00:00:00
Title: UK economy returns to growth in May, beating expectations as British pound hits four-month high
Content: City of London skyline on 10th June 2024 in London, United Kingdom. The City of London is a city, ceremonial county and local government district that contains the primary central business district CB...

Date: 2024-06-07 00:00:00
Title: War, what war? Putin looks to woo new business partners willing to overlook its invasion of Ukraine
Content: Russian President Vladimir Putin is seen while visiting the Lakhta Center on June 5, 2024, in Saint Petersburg, Russia. Vladimir Putin visited a newly built Lakhta Center, a skyscraper of Gazprom,

In [ ]:
df = pd.DataFrame(collected_articles)

In [ ]:
df.head()

,date,title,text
0,2024-05-25,The Fed probably won't be delivering any inter...,Traders work on the floor of the New York Stoc...
1,2024-07-11,"UK economy returns to growth in May, beating e...",City of London skyline on 10th June 2024 in Lo...
2,2024-06-07,"War, what war? Putin looks to woo new business...",Russian President Vladimir Putin is seen while...
3,2024-03-12,Consumer prices rose 0.4% in February and 3.2%...,"watch now\n\nInflation rose again in February,..."
4,2024-04-26,Key Fed inflation measure rose 2.8% in March f...,Inflation showed few signs of letting up in Ma...


In [ ]:
df.tail()

,date,title,text
196,2024-03-07,European Central Bank hints at June rate cut a...,The European Central Bank on Thursday lowered ...
197,2024-04-10,Hot inflation data pushes market’s rate cut ex...,Traders work on the floor of the New York Stoc...
198,2024-04-02,"Steve Eisman says the Fed shouldn't cut rates,...",If the Federal Reserve follows through on plan...
199,2024-06-28,Financial crimes watchdog removes Turkey from ...,"As the sunsets, a ferry boat glides across the..."
200,2024-06-14,"Bank of Japan set to reduce JGB purchases, sta...",The Bank of Japan is largely expected to hold ...


In [ ]:
# 날짜 순으로 정렬
df2 = df.sort_values(by='date')

In [ ]:
df2.head()

,date,title,text
152,2024-01-31,"Private payroll growth slowed to just 107,000 ...",Private payroll growth declined sharply in Jan...
26,2024-02-01,"Euro zone inflation eases as expected, but cor...",Patrons at sidewalk tables of Janis bar in Cai...
143,2024-02-01,January hiring was the lowest for the month on...,Companies announced the highest level of job c...
188,2024-02-02,"U.S. economy added 353,000 jobs in January, mu...",watch now\n\nJob growth posted a surprisingly ...
162,2024-02-02,The unemployment rate of Black men rose in Jan...,A networking and hiring event for professional...


In [ ]:
df2.tail()

,date,title,text
29,2024-07-16,Trump says he would let Fed Chair Powell finis...,Republican presidential nominee and former U.S...
25,2024-07-16,IMF upgrades UK growth forecast in another boo...,Seen through the branches of trees in Ruskin P...
142,2024-07-16,IMF sees ‘bumps’ in path to lower inflation,The International Monetary Fund warned Tuesday...
36,2024-07-17,Wealth in Turkey grew the most in the world at...,Turkey came out miles ahead of the rest of the...
88,2024-07-17,UK inflation holds steady at Bank of England's...,Alexander Spatari | Moment | Getty Images\n\nU...


In [ ]:
# 엑셀 파일을 Google Drive의 특정 경로에 저장
excel_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_Economy.xlsx'
df2.to_excel(excel_path, index=False)

3. FINANCE

In [ ]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.get('https://www.cnbc.com/finance/') # 브라우저 불러오기

In [ ]:
# Load More 버튼을 반복적으로 클릭하여 더 많은 기사 로드
while True:
    try:
        # iframe 숨기기
        iframes = browser.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            browser.execute_script("arguments[0].style.visibility='hidden';", iframe)

        # 페이지 스크롤
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Load More 버튼이 나타날 때까지 대기
        load_more_button = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'icon-arrow-down-readmore'))
        )

        # ActionChains를 사용하여 버튼이 가려지지 않도록 조정
        actions = ActionChains(browser)
        actions.move_to_element(load_more_button).perform()
        time.sleep(2)  # 페이지 로드 대기
        load_more_button.click()
        time.sleep(5)  # 페이지 로드 대기
    except Exception as e:
        print(f"No more articles to load or an error occurred: {e}")
        break
print('All articles loaded.')

No more articles to load or an error occurred: Message: 
Stacktrace:
#0 0x5a8544ae5c5a <unknown>
#1 0x5a85447c8e2c <unknown>
#2 0x5a8544815661 <unknown>
#3 0x5a8544815751 <unknown>
#4 0x5a8544859f64 <unknown>
#5 0x5a85448385ed <unknown>
#6 0x5a8544857303 <unknown>
#7 0x5a8544838363 <unknown>
#8 0x5a8544808247 <unknown>
#9 0x5a8544808b9e <unknown>
#10 0x5a8544aac22b <unknown>
#11 0x5a8544ab02d1 <unknown>
#12 0x5a8544a97ade <unknown>
#13 0x5a8544ab0e32 <unknown>
#14 0x5a8544a7c77f <unknown>
#15 0x5a8544ad5618 <unknown>
#16 0x5a8544ad57f0 <unknown>
#17 0x5a8544ae4d8c <unknown>
#18 0x7fc90afa1ac3 <unknown>

All articles loaded.


In [ ]:
# URL 수집을 위한 BeautifulSoup 사용
soup = BeautifulSoup(browser.page_source, 'html.parser')

# URL 리스트 불러오기
url_lst = []
contents = soup.find_all('a', class_='Card-title')
for content in contents:
    url_lst.append(content['href'])
url_lst = list(set(url_lst))  # 중복 URL 제거
print(f"Total URLs collected: {len(url_lst)}")

Total URLs collected: 201


In [ ]:
url_lst[:]

['https://www.cnbc.com/2024/06/16/investors-may-be-missing-these-commercial-real-estate-opportunities.html',
 'https://www.cnbc.com/2024/07/16/bank-of-america-bac-earnings-q2-2024.html',
 'https://www.cnbc.com/2024/07/11/chinas-robotaxi-push-sparks-concerns-about-job-security-for-drivers.html',
 'https://www.cnbc.com/2024/06/21/china-spent-230-billion-to-build-its-electric-car-industry-csis-says.html',
 'https://www.cnbc.com/2024/06/29/growth-value-stocks-could-see-boost-from-russell-rebalancing-.html',
 'https://www.cnbc.com/2024/06/12/roaring-kitty-has-some-tough-choices-to-make-on-his-gamestop-options-.html',
 'https://www.cnbc.com/2024/07/11/high-inflation-is-largely-not-bidens-or-trumps-fault-economists-say.html',
 'https://www.cnbc.com/2024/07/01/ken-griffin-says-hes-not-convinced-ai-will-replace-human-jobs-in-near-future.html',
 'https://www.cnbc.com/2024/07/16/traders-see-the-odds-of-a-fed-rate-cut-by-september-at-100percent.html',
 'https://www.cnbc.com/2024/06/20/stocks-makin

In [ ]:
# 브라우저 종료
browser.quit()

In [ ]:
# 기사 수집 함수
def get_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "text": article.text,
        "publish_date": article.publish_date
    }

In [ ]:
# 기사 수집
collected_articles = []

In [ ]:
for url in url_lst:
    try:
        content = get_article_content(url)
        publish_date = content["publish_date"]

        # 기사 데이터 추가
        collected_articles.append({
            "date": publish_date,
            "title": content["title"],
            "text": content["text"]
        })
    except Exception as e:
        print(f"Failed to process article at {url}: {e}")

In [ ]:
# 수집된 기사 출력
for article in collected_articles:
    print(f"Date: {article['date']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['text'][:200]}...")  # 본문은 길기 때문에 앞부분만 출력
    print()

Date: 2024-06-16 00:00:00
Title: Investors may be looking at commercial real estate risk all wrong and missing these opportunities
Content: The threat from a looming wave of maturing commercial real estate loans has been well telegraphed to investors, but it's possible the metrics they are using to protect themselves from risk are flawed....

Date: 2024-07-16 00:00:00
Title: Bank of America shares jump 5% after saying net interest income rebound is coming
Content: Bank of America on Tuesday said second-quarter revenue and profit topped expectations on rising investment banking and asset management fees.

Here's what the company reported:

Earnings: 83 cents a s...

Date: 2024-07-11 00:00:00
Title: China's robotaxi push sparks concerns about job security for drivers
Content: More than 70% of Baidu Apollo Go robotaxi rides in Wuhan were fully driverless as of April, and the company said in May it expected 100% of the rides to be completely autonomous in coming quarters. Bl...

Date: 2024

In [ ]:
df = pd.DataFrame(collected_articles)

In [ ]:
df.head()

,date,title,text
0,2024-06-16,Investors may be looking at commercial real es...,The threat from a looming wave of maturing com...
1,2024-07-16,Bank of America shares jump 5% after saying ne...,Bank of America on Tuesday said second-quarter...
2,2024-07-11,China's robotaxi push sparks concerns about jo...,More than 70% of Baidu Apollo Go robotaxi ride...
3,2024-06-21,China has spent at least $230 billion to build...,"Workers assemble a Wuling Hongguang Mini EV, a..."
4,2024-06-29,"Growth, value stocks could see boost from Russ...",A bullish move may be ahead for both value and...


In [ ]:
df.tail()

,date,title,text
196,2024-06-05,Hanesbrands to sell Champion brand to Authenti...,Hanesbrands agreed to sell its global Champion...
197,2024-06-21,China talks up support for IPOs. Investors are...,A man walks a dog in the shade away from the m...
198,2024-07-03,'Early innings' of a U.S. manufacturing boom: ...,One exchange-traded fund is betting on a U.S. ...
199,2024-06-11,Stocks making the biggest moves premarket: Eli...,Check out the companies making headlines befor...
200,2024-07-12,Here's the deflation breakdown for June 2024 —...,97 / Getty\n\nInflation has throttled back sig...


In [ ]:
# 날짜 순으로 정렬
df2 = df.sort_values(by='date')

In [ ]:
df2.head()

,date,title,text
91,2024-05-21,Navigate 2024 with these 5 insightful consumer...,
142,2024-06-04,Fintech firm Nium cuts valuation by 30% in new...,Westend61 | Westend61 | Getty Images\n\nAMSTER...
97,2024-06-04,Stocks making the biggest moves after hours: C...,Check out the companies making headlines in af...
43,2024-06-05,Citadel Securities and BlackRock back project ...,The Texas flag is seen before the game between...
170,2024-06-05,Stocks making the biggest moves after hours: L...,Check out the companies making headlines after...


In [ ]:
df2.tail()

,date,title,text
88,2024-07-17,HSBC appoints Georges Elhedery as group CEO st...,HSBC logo is displayed outside a branch of in ...
193,2024-07-17,Stocks making the biggest moves after hours: U...,Check out the companies making headlines in af...
18,2024-07-18,Stocks making the biggest moves premarket: Dom...,Check out the companies making headlines befor...
65,2024-07-18,JD Vance blames U.S. wage losses on China's ef...,Relations between Washington and Beijing are a...
166,2024-07-18,Watch: ECB President Christine Lagarde speaks ...,[The stream is slated to start at 8:45 a.m. ET...


In [ ]:
# 엑셀 파일을 Google Drive의 특정 경로에 저장
excel_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_Finance.xlsx'
df2.to_excel(excel_path, index=False)

4. HEALTH AND SCIENCE

In [ ]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.get('https://www.cnbc.com/health-and-science/') # 브라우저 불러오기

In [ ]:
# Load More 버튼을 반복적으로 클릭하여 더 많은 기사 로드
while True:
    try:
        # iframe 숨기기
        iframes = browser.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            browser.execute_script("arguments[0].style.visibility='hidden';", iframe)

        # 페이지 스크롤
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Load More 버튼이 나타날 때까지 대기
        load_more_button = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'icon-arrow-down-readmore'))
        )

        # ActionChains를 사용하여 버튼이 가려지지 않도록 조정
        actions = ActionChains(browser)
        actions.move_to_element(load_more_button).perform()
        time.sleep(2)  # 페이지 로드 대기
        load_more_button.click()
        time.sleep(5)  # 페이지 로드 대기
    except Exception as e:
        print(f"No more articles to load or an error occurred: {e}")
        break
print('All articles loaded.')

No more articles to load or an error occurred: Message: 
Stacktrace:
#0 0x5582eea2ac5a <unknown>
#1 0x5582ee70de2c <unknown>
#2 0x5582ee75a661 <unknown>
#3 0x5582ee75a751 <unknown>
#4 0x5582ee79ef64 <unknown>
#5 0x5582ee77d5ed <unknown>
#6 0x5582ee79c303 <unknown>
#7 0x5582ee77d363 <unknown>
#8 0x5582ee74d247 <unknown>
#9 0x5582ee74db9e <unknown>
#10 0x5582ee9f122b <unknown>
#11 0x5582ee9f52d1 <unknown>
#12 0x5582ee9dcade <unknown>
#13 0x5582ee9f5e32 <unknown>
#14 0x5582ee9c177f <unknown>
#15 0x5582eea1a618 <unknown>
#16 0x5582eea1a7f0 <unknown>
#17 0x5582eea29d8c <unknown>
#18 0x7f79348a7ac3 <unknown>

All articles loaded.


In [ ]:
# URL 수집을 위한 BeautifulSoup 사용
soup = BeautifulSoup(browser.page_source, 'html.parser')

# URL 리스트 불러오기
url_lst = []
contents = soup.find_all('a', class_='Card-title')
for content in contents:
    url_lst.append(content['href'])
url_lst = list(set(url_lst))  # 중복 URL 제거
print(f"Total URLs collected: {len(url_lst)}")

Total URLs collected: 201


In [ ]:
url_lst[:]

['https://www.cnbc.com/2024/05/31/pfizer-lung-cancer-drug-shows-promising-long-term-trial-results.html',
 'https://www.cnbc.com/2024/05/28/why-walmart-walgreens-cvs-health-clinic-experiment-is-struggling.html',
 'https://www.cnbc.com/2024/07/11/healthy-returns-drugmakers-are-racing-to-develop-weight-loss-pills.html',
 'https://www.cnbc.com/2024/05/14/problems-that-rich-people-face-according-to-therapists-.html',
 'https://www.cnbc.com/2024/04/16/johnson-johnson-jnj-earnings-q1-2024.html',
 'https://www.cnbc.com/2024/04/08/trump-says-abortion-restrictions-should-be-left-to-states-dodging-a-national-ban.html',
 'https://www.cnbc.com/2024/04/02/extreme-heat-from-miami-to-melbourne-a-quiet-revolution-is-underway.html',
 'https://www.cnbc.com/2024/03/24/nvidias-ai-ambitions-in-medicine-and-health-care-are-becoming-clear.html',
 'https://www.cnbc.com/2024/06/27/supreme-court-blocks-purdue-pharma-opioid-settlement.html',
 'https://www.cnbc.com/2024/04/30/change-healthcare-cyberattack-doctors-

In [ ]:
# 브라우저 종료
browser.quit()

In [ ]:
# 기사 수집 함수
def get_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "text": article.text,
        "publish_date": article.publish_date
    }

In [ ]:
# 기사 수집
collected_articles = []

In [ ]:
for url in url_lst:
    try:
        content = get_article_content(url)
        publish_date = content["publish_date"]

        # 기사 데이터 추가
        collected_articles.append({
            "date": publish_date,
            "title": content["title"],
            "text": content["text"]
        })
    except Exception as e:
        print(f"Failed to process article at {url}: {e}")

In [ ]:
# 수집된 기사 출력
for article in collected_articles:
    print(f"Date: {article['date']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['text'][:200]}...")  # 본문은 길기 때문에 앞부분만 출력
    print()

Date: 2024-05-31 00:00:00
Title: Pfizer's drug for advanced lung cancer shows promising long-term trial results
Content: CHICAGO — Pfizer on Friday said its drug for an advanced form of lung cancer showed promising long-term results in a late-stage trial, which may help establish it as the new standard treatment for the...

Date: 2024-05-28 00:00:00
Title: Why Walmart, Walgreens, CVS retail health clinic experiment is struggling
Content: Bobbi Radford showed up at the CVS MinuteClinic in Batavia, Ohio, last Thanksgiving because she had pain in her arm.

"I waited an hour and then was told to go to the [emergency room].," Radford said....

Date: 2024-07-11 00:00:00
Title: Healthy Returns: Drugmakers are racing to develop more convenient weight loss pills
Content: Think a friend or colleague should be getting this newsletter? Share this link with them to sign up.

Good afternoon! Several drugmakers are hurrying to capitalize on one of the next major innovations...

Date: 2024-05-14 00:00

In [ ]:
df = pd.DataFrame(collected_articles)

In [ ]:
df.head()

,date,title,text
0,2024-05-31,Pfizer's drug for advanced lung cancer shows p...,CHICAGO — Pfizer on Friday said its drug for a...
1,2024-05-28,"Why Walmart, Walgreens, CVS retail health clin...",Bobbi Radford showed up at the CVS MinuteClini...
2,2024-07-11,Healthy Returns: Drugmakers are racing to deve...,Think a friend or colleague should be getting ...
3,2024-05-14,'Wealth can be pretty isolating': Problems tha...,"Depressed young male smoking cigarette, drinki..."
4,2024-04-16,Johnson & Johnson tops quarterly profit estima...,An entry sign to the Johnson & Johnson campus ...


In [ ]:
df.tail()

,date,title,text
196,2024-05-10,Novavax shares nearly double on Sanofi deal to...,A health worker prepares a dose of the Novavax...
197,2024-05-21,AstraZeneca targets $80 billion in total reven...,In this article NOVO.B-DK\n\nAZN-GB\n\nLLY Fol...
198,2024-04-17,"Most doses of Eli Lilly's Zepbound, Mounjaro i...","An injection pen of Zepbound, Eli Lilly’s weig..."
199,2024-04-26,FDA approves Pfizer’s first gene therapy for r...,The Food and Drug Administration on Friday ann...
200,2024-04-03,"Healthy Returns: Weight loss, diabetes drug ad...",Boxes of Wegovy made by Novo Nordisk are seen ...


In [ ]:
# 날짜 순으로 정렬
df2 = df.sort_values(by='date')

In [ ]:
df2.head()

,date,title,text
191,2024-03-19,"Nvidia stomps the bears. Plus, Cramer's though...","Every weekday, the CNBC Investing Club with Ji..."
95,2024-03-19,Jim Cramer defends Elon Musk's use of ketamine...,CNBC's Jim Cramer said Tuesday that Elon Musk'...
27,2024-03-20,FDA says safety issues with China-made syringe...,The U.S. Food and Drug Administration's Center...
51,2024-03-20,Neuralink shares video of patient playing ches...,The Neuralink logo is displayed on a smartphon...
21,2024-03-21,We're buying more of this health-care company ...,"Shortly after the opening bell, we will be buy..."


In [ ]:
df2.tail()

,date,title,text
116,2024-07-16,Brain implant patient says OpenAI's tech helps...,Mark using Synchron's BCI Courtesy: Synchron\n...
117,2024-07-16,Healthy Returns: Diabetes drug Ozempic may low...,A box of Ozempic made by Novo Nordisk is seen ...
24,2024-07-17,New Mexico attorney general launches probe of ...,New Mexico Attorney General Raúl Torrez discus...
77,2024-07-17,A growing number of parents are refusing to gi...,Photo: Jaromir Chalabala / Getty\n\nLONDON — T...
48,2024-07-18,Op-ed: How well can AI chatbots mimic doctors ...,"Hands, tablet and doctor with body hologram, o..."


In [ ]:
# 엑셀 파일을 Google Drive의 특정 경로에 저장
excel_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_HealthandScience.xlsx'
df2.to_excel(excel_path, index=False)

5. REAL ESTATE

In [ ]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.get('https://www.cnbc.com/real-estate/') # 브라우저 불러오기

In [ ]:
# Load More 버튼을 반복적으로 클릭하여 더 많은 기사 로드
while True:
    try:
        # iframe 숨기기
        iframes = browser.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            browser.execute_script("arguments[0].style.visibility='hidden';", iframe)

        # 페이지 스크롤
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Load More 버튼이 나타날 때까지 대기
        load_more_button = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'icon-arrow-down-readmore'))
        )

        # ActionChains를 사용하여 버튼이 가려지지 않도록 조정
        actions = ActionChains(browser)
        actions.move_to_element(load_more_button).perform()
        time.sleep(2)  # 페이지 로드 대기
        load_more_button.click()
        time.sleep(5)  # 페이지 로드 대기
    except Exception as e:
        print(f"No more articles to load or an error occurred: {e}")
        break
print('All articles loaded.')

No more articles to load or an error occurred: Message: 
Stacktrace:
#0 0x56ee1e818c5a <unknown>
#1 0x56ee1e4fbe2c <unknown>
#2 0x56ee1e548661 <unknown>
#3 0x56ee1e548751 <unknown>
#4 0x56ee1e58cf64 <unknown>
#5 0x56ee1e56b5ed <unknown>
#6 0x56ee1e58a303 <unknown>
#7 0x56ee1e56b363 <unknown>
#8 0x56ee1e53b247 <unknown>
#9 0x56ee1e53bb9e <unknown>
#10 0x56ee1e7df22b <unknown>
#11 0x56ee1e7e32d1 <unknown>
#12 0x56ee1e7caade <unknown>
#13 0x56ee1e7e3e32 <unknown>
#14 0x56ee1e7af77f <unknown>
#15 0x56ee1e808618 <unknown>
#16 0x56ee1e8087f0 <unknown>
#17 0x56ee1e817d8c <unknown>
#18 0x7d31d1682ac3 <unknown>

All articles loaded.


In [ ]:
# URL 수집을 위한 BeautifulSoup 사용
soup = BeautifulSoup(browser.page_source, 'html.parser')

# URL 리스트 불러오기
url_lst = []
contents = soup.find_all('a', class_='Card-title')
for content in contents:
    url_lst.append(content['href'])
url_lst = list(set(url_lst))  # 중복 URL 제거
print(f"Total URLs collected: {len(url_lst)}")

Total URLs collected: 201


In [ ]:
url_lst[:]

['https://www.cnbc.com/2024/05/30/37percent-of-americans-paid-a-late-fee-in-the-last-12-months-report-finds.html',
 'https://www.cnbc.com/2024/05/10/here-are-key-things-to-know-before-you-buy-a-house-abroad.html',
 'https://www.cnbc.com/2024/06/13/9-stocks-that-can-benefit-from-fed-interest-rate-cuts.html',
 'https://www.cnbc.com/video/2024/04/17/property-brothers-if-youre-going-to-buy-a-house-get-something-you-know-your-family-can-grow-into.html',
 'https://www.cnbc.com/video/2024/05/30/aprils-pending-home-sales-fall-7-point-7-percent-monthly-as-higher-rates-dampen-demand.html',
 'https://www.cnbc.com/2024/06/10/heres-what-to-expect-from-the-housing-market-in-the-second-half-of-2024.html',
 'https://www.cnbc.com/2024/07/13/with-high-prices-and-mortgage-rates-homeowners-feel-stuck.html',
 'https://www.cnbc.com/2024/07/12/china-to-hold-third-plenum-why-real-estate-isnt-likely-the-main-focus.html',
 'https://www.cnbc.com/video/2024/05/28/doesnt-appear-to-be-relief-on-the-horizon-as-far-a

In [ ]:
# 브라우저 종료
browser.quit()

In [ ]:
# 기사 수집 함수
def get_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "text": article.text,
        "publish_date": article.publish_date
    }

In [ ]:
# 기사 수집
collected_articles = []

In [ ]:
for url in url_lst:
    try:
        content = get_article_content(url)
        publish_date = content["publish_date"]

        # 기사 데이터 추가
        collected_articles.append({
            "date": publish_date,
            "title": content["title"],
            "text": content["text"]
        })
    except Exception as e:
        print(f"Failed to process article at {url}: {e}")

In [ ]:
# 수집된 기사 출력
for article in collected_articles:
    print(f"Date: {article['date']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['text'][:200]}...")  # 본문은 길기 때문에 앞부분만 출력
    print()

Date: 2024-05-30 00:00:00
Title: 37% of Americans paid a late fee in the last 12 months, report finds
Content: Blackcat | E+ | Getty Images

'Speak directly to your creditors'

If you are beginning to fall behind on usual monthly payments or anticipate doing so, it's best to "speak directly to your creditors b...

Date: 2024-05-10 00:00:00
Title: What to know before you buy a house overseas — and 3 steps to smooth the process
Content: "Living in a country is not the same as spending a lovely three weeks there," said certified financial planner Jude Boudreaux, a partner and senior financial planner at The Planning Center in New Orle...

Date: 2024-06-13 00:00:00
Title: Here are 9 stocks that can benefit from Fed interest rate cuts
Content: The S & P 500 and Nasdaq extended their record rallies this week following cooler-than-expected consumer inflation data Wednesday morning. While Fed rate cuts would likely benefit the overall stock ma...

Date: 2024-04-17 00:00:00
Title: Property Brot

In [ ]:
df = pd.DataFrame(collected_articles)

In [ ]:
df.head()

,date,title,text
0,2024-05-30,37% of Americans paid a late fee in the last 1...,Blackcat | E+ | Getty Images\n\n'Speak directl...
1,2024-05-10,What to know before you buy a house overseas —...,"""Living in a country is not the same as spendi..."
2,2024-06-13,Here are 9 stocks that can benefit from Fed in...,The S & P 500 and Nasdaq extended their record...
3,2024-04-17,Property Brothers: If you're going to buy a ho...,Share Share Article via Facebook Share Article...
4,2024-05-30,April's pending home sales fall 7.7% monthly a...,Share Share Article via Facebook Share Article...


In [ ]:
df.tail()

,date,title,text
196,2024-05-07,Real Estate Mogul Barry Sternlicht: Migrants a...,Share Share Article via Facebook Share Article...
197,2024-06-24,Housing in pandemic boom towns up to 40% overv...,Share Share Article via Facebook Share Article...
198,2024-05-17,Beijing is acknowledging the existing property...,Share Share Article via Facebook Share Article...
199,2024-06-18,Office space warning: Activist Investor Jonath...,Share Share Article via Facebook Share Article...
200,2024-06-27,Vienna is the world's most livable city for th...,Share Share Article via Facebook Share Article...


In [ ]:
# 날짜 순으로 정렬
df2 = df.sort_values(by='date')

In [ ]:
df2.head()

,date,title,text
57,2022-02-17,Understanding the value of branded content,
92,2024-04-16,Regional bank earnings may expose critical wea...,Regional bank earnings may expose critical wea...
3,2024-04-17,Property Brothers: If you're going to buy a ho...,Share Share Article via Facebook Share Article...
125,2024-04-17,Morgan Stanley's Lauren Hochfelder: There's bi...,Share Share Article via Facebook Share Article...
11,2024-04-17,Why mortgage applications have jumped despite ...,Share Share Article via Facebook Share Article...


In [ ]:
df2.tail()

,date,title,text
128,2024-07-17,Rent caps cause unintended consequences for af...,Share Share Article via Facebook Share Article...
164,2024-07-17,China won't stimulate its economy until bigger...,Share Share Article via Facebook Share Article...
69,2024-07-17,How on-time rent payments can help 'credit inv...,watch now\n\nHousing is the most considerable ...
163,2024-07-18,The consumer is struggling with home affordabi...,Share Share Article via Facebook Share Article...
153,2024-07-18,Some renters may be ‘mortgage-ready’ and not k...,Are you ready to buy a home? Many renters have...


In [ ]:
# 엑셀 파일을 Google Drive의 특정 경로에 저장
excel_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_RealEstate.xlsx'
df2.to_excel(excel_path, index=False)

6. ENERGY

In [ ]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.get('https://www.cnbc.com/energy/') # 브라우저 불러오기

In [ ]:
# Load More 버튼을 반복적으로 클릭하여 더 많은 기사 로드
while True:
    try:
        # iframe 숨기기
        iframes = browser.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            browser.execute_script("arguments[0].style.visibility='hidden';", iframe)

        # 페이지 스크롤
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Load More 버튼이 나타날 때까지 대기
        load_more_button = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'icon-arrow-down-readmore'))
        )

        # ActionChains를 사용하여 버튼이 가려지지 않도록 조정
        actions = ActionChains(browser)
        actions.move_to_element(load_more_button).perform()
        time.sleep(2)  # 페이지 로드 대기
        load_more_button.click()
        time.sleep(5)  # 페이지 로드 대기
    except Exception as e:
        print(f"No more articles to load or an error occurred: {e}")
        break
print('All articles loaded.')

No more articles to load or an error occurred: Message: 
Stacktrace:
#0 0x58c672417c5a <unknown>
#1 0x58c6720fae2c <unknown>
#2 0x58c672147661 <unknown>
#3 0x58c672147751 <unknown>
#4 0x58c67218bf64 <unknown>
#5 0x58c67216a5ed <unknown>
#6 0x58c672189303 <unknown>
#7 0x58c67216a363 <unknown>
#8 0x58c67213a247 <unknown>
#9 0x58c67213ab9e <unknown>
#10 0x58c6723de22b <unknown>
#11 0x58c6723e22d1 <unknown>
#12 0x58c6723c9ade <unknown>
#13 0x58c6723e2e32 <unknown>
#14 0x58c6723ae77f <unknown>
#15 0x58c672407618 <unknown>
#16 0x58c6724077f0 <unknown>
#17 0x58c672416d8c <unknown>
#18 0x7dbafb3ecac3 <unknown>

All articles loaded.


In [ ]:
# URL 수집을 위한 BeautifulSoup 사용
soup = BeautifulSoup(browser.page_source, 'html.parser')

# URL 리스트 불러오기
url_lst = []
contents = soup.find_all('a', class_='Card-title')
for content in contents:
    url_lst.append(content['href'])
url_lst = list(set(url_lst))  # 중복 URL 제거
print(f"Total URLs collected: {len(url_lst)}")

Total URLs collected: 201


In [ ]:
url_lst[:]

['https://www.cnbc.com/2024/06/19/solar-is-growing-faster-than-any-energy-source-as-clean-power-for-data-centers.html',
 'https://www.cnbc.com/2024/05/08/siemens-energy-changes-leadership-at-embattled-wind-turbine-unit-.html',
 'https://www.cnbc.com/2024/04/26/climate-exxon-working-on-direct-air-capture-but-costs-need-to-fall.html',
 'https://www.cnbc.com/2024/06/04/politics-playing-a-role-in-ev-adoption-michigan-gov-whitmer.html',
 'https://www.cnbc.com/2024/07/19/solar-stocks-tumble-as-clean-energy-investors-fear-a-trump-win.html',
 'https://www.cnbc.com/2024/07/17/ethereum-co-founder-warns-against-voting-only-on-pro-crypto.html',
 'https://www.cnbc.com/2024/06/24/crude-oil-prices-today.html',
 'https://www.cnbc.com/2024/05/06/exxon-says-chevron-dispute-over-hess-guyana-assets-could-drag-into-2025.html',
 'https://www.cnbc.com/2024/05/10/jim-cramer-likes-akamai-for-a-hidden-jewel-not-for-content-delivery.html',
 'https://www.cnbc.com/2024/06/05/opec-shift-casts-a-bearish-shadow-over-

In [ ]:
# 브라우저 종료
browser.quit()

In [ ]:
# 기사 수집 함수
def get_article_content(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "text": article.text,
        "publish_date": article.publish_date
    }

In [ ]:
# 기사 수집
collected_articles = []

In [ ]:
for url in url_lst:
    try:
        content = get_article_content(url)
        publish_date = content["publish_date"]

        # 기사 데이터 추가
        collected_articles.append({
            "date": publish_date,
            "title": content["title"],
            "text": content["text"]
        })
    except Exception as e:
        print(f"Failed to process article at {url}: {e}")

In [ ]:
# 수집된 기사 출력
for article in collected_articles:
    print(f"Date: {article['date']}")
    print(f"Title: {article['title']}")
    print(f"Content: {article['text'][:200]}...")  # 본문은 길기 때문에 앞부분만 출력
    print()

Date: 2024-06-19 00:00:00
Title: Solar is growing faster than any electricity source as Big Tech seeks clean energy for data centers
Content: Aerial view of the Oberon Solar O&M farm on March 24, 2024 in Ector County, Texas. In 2023, Texas led the nation in solar installations on its power grid, surpassing California for its second consecut...

Date: 2024-05-08 00:00:00
Title: Siemens Energy shares jump 13% after guidance raise and leadership change at embattled wind turbine unit
Content: Siemens Energy shares soared as much as 13% on Wednesday after the German renewables firm raised its forecast for the year and announced that the CEO of its troubled wind turbine unit will be replaced...

Date: 2024-04-26 00:00:00
Title: Exxon is working on tech to remove CO2 from atmosphere but a breakthrough is needed to slash costs
Content: Exxon Mobil is working on technology to directly remove carbon dioxide from the atmosphere with the goal of slashing sky-high costs by half, CEO Darren Woods sa

In [ ]:
df = pd.DataFrame(collected_articles)

In [ ]:
df.head()

,date,title,text
0,2024-06-19,Solar is growing faster than any electricity s...,Aerial view of the Oberon Solar O&M farm on Ma...
1,2024-05-08,Siemens Energy shares jump 13% after guidance ...,Siemens Energy shares soared as much as 13% on...
2,2024-04-26,Exxon is working on tech to remove CO2 from at...,Exxon Mobil is working on technology to direct...
3,2024-06-04,Politics playing a role in electric vehicle ad...,The future of the electric vehicle is a critic...
4,2024-07-19,Solar stocks tumble as clean energy investors ...,Solar stocks tumbled this week as Donald Trump...


In [ ]:
df.tail()

,date,title,text
196,2024-05-20,New York AG announces $2 billion settlement wi...,New York Attorney General Letitia James is set...
197,2024-07-11,Here's the inflation breakdown for June 2024 —...,David Paul Morris/Bloomberg via Getty Images\n...
198,2024-05-01,"Exxon Mobil reaches agreement with FTC, poised...",The Federal Trade Commission will wave through...
199,2024-05-13,AI hype is lifting these dividend payers. Thes...,"The once-sleepy utilities sector, known best f..."
200,2024-07-01,Collapsed bitcoin exchange Mt. Gox is about to...,Omer Taha Cetin | Anadolu | Getty Images\n\nA ...


In [ ]:
# 날짜 순으로 정렬
df2 = df.sort_values(by='date')

In [ ]:
df2.head()

,date,title,text
14,2024-04-26,"Dividend stocks are poised to outperform, says...",The U.S. economy is flashing a sign that's fav...
2,2024-04-26,Exxon is working on tech to remove CO2 from at...,Exxon Mobil is working on technology to direct...
36,2024-04-26,Exxon stock falls as earnings miss on lower na...,An Exxon gas station is seen in the Brooklyn b...
173,2024-04-27,‘This is a unique time’: ARK Invest’s chief fu...,ARK Invest's chief futurist lists five groups ...
146,2024-04-29,Oil prices fall more than 1% as Secretary of S...,U.S. Secretary of State Antony Blinken attends...


In [ ]:
df2.tail()

,date,title,text
5,2024-07-17,Ethereum co-founder warns against voting only ...,Ethereum co-creator Vitalik Buterin on Wednesd...
91,2024-07-18,"Bye-bye bitcoin, hello AI: Texas miners leave ...",Cattle graze at the Buffalo Gap Wind Power pro...
162,2024-07-18,Trump Bitcoin Conference fundraiser tickets to...,Former U.S. President and Republican president...
76,2024-07-18,U.S. oil holds to gains after three weeks of c...,U.S. oil futures on Thursday held to gains aft...
4,2024-07-19,Solar stocks tumble as clean energy investors ...,Solar stocks tumbled this week as Donald Trump...


In [ ]:
# 엑셀 파일을 Google Drive의 특정 경로에 저장
excel_path = '/content/drive/MyDrive/Colab Notebooks/미래에셋/CNBC_articles_Energy.xlsx'
df2.to_excel(excel_path, index=False)